# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet'])
from sqlalchemy import create_engine
import re
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [13]:
# load data from database
engine = create_engine('sqlite:///etl_message.db')
df = pd.read_sql_table("etl_message", con=engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)
category_names = Y.columns

In [3]:
X

0        Weather update - a cold front from Cuba that c...
1                  Is the Hurricane over or is it not over
2                          Looking for someone but no name
3        UN reports Leogane 80-90 destroyed. Only Hospi...
4        says: west side of Haiti, rest of the country ...
5                   Information about the National Palace-
6                           Storm at sacred heart of jesus
7        Please, we need tents and water. We are in Sil...
8          I would like to receive the messages, thank you
9        I am in Croix-des-Bouquets. We have health iss...
10       There's nothing to eat and water, we starving ...
11       I am in Petionville. I need more information r...
12       I am in Thomassin number 32, in the area named...
13       Let's do it together, need food in Delma 75, i...
14       More information on the 4636 number in order f...
15       A Comitee in Delmas 19, Rue ( street ) Janvier...
16       We need food and water in Klecin 12. We are dy.

### 2. Write a tokenization function to process your text data

In [6]:
def tokenize(text):
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [7]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, Y)

pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
def class_rep(model, X_test, y_test, category_names):
    y_pred = model.predict(X_test)
    for i, col in enumerate(category_names):
        print(col)
        print(classification_report(y_test[col], y_pred[:, i]))

In [14]:
class_rep(pipeline, X_test, y_test, category_names)

related
             precision    recall  f1-score   support

          0       0.63      0.35      0.45      1514
          1       0.82      0.94      0.88      4994
          2       0.64      0.15      0.25        46

avg / total       0.78      0.80      0.77      6554

request
             precision    recall  f1-score   support

          0       0.88      0.98      0.93      5460
          1       0.81      0.36      0.50      1094

avg / total       0.87      0.88      0.86      6554

offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6525
          1       0.00      0.00      0.00        29

avg / total       0.99      1.00      0.99      6554

aid_related
             precision    recall  f1-score   support

          0       0.72      0.89      0.79      3823
          1       0.76      0.51      0.61      2731

avg / total       0.74      0.73      0.72      6554

medical_help
             precision    recall  f1-sco

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [9]:
parameters = {'tfidf__use_idf':[True, False],
              'clf__estimator__n_estimators': [50, 100],
              'clf__estimator__min_samples_split': [2, 4]}

cv = GridSearchCV(pipeline, param_grid=parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [10]:
cv.fit(X_train, y_train)
class_rep(cv, X_test, y_test)

c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


related
              precision    recall  f1-score   support

           0       0.73      0.26      0.39      1564
           1       0.80      0.97      0.88      4947
           2       0.67      0.09      0.16        43

   micro avg       0.80      0.80      0.80      6554
   macro avg       0.73      0.44      0.48      6554
weighted avg       0.78      0.80      0.76      6554

request
              precision    recall  f1-score   support

           0       0.90      0.99      0.94      5445
           1       0.89      0.44      0.59      1109

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.89      0.71      0.76      6554
weighted avg       0.89      0.90      0.88      6554

offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0.00      0.00      0.00        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.50      0.50      0.50      655

c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.00      0.00      0.00        75

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.49      0.50      0.50      6554
weighted avg       0.98      0.99      0.98      6554

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6267
           1       0.00      0.00      0.00       287

   micro avg       0.96      0.96      0.96      6554
   macro avg       0.48      0.50      0.49      6554
weighted avg       0.91      0.96      0.93      6554

weather_related
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      4735
           1       0.88      0.63      0.74      1819

   micro avg       0.87      0.87      0.87      6554
   macro avg       0.88      0.80      0.83      6554
weighted avg       0.87      0.87    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [11]:
pipeline2 = Pipeline([
        ('vect', CountVectorizer(tokenizer = tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [12]:
pipeline2.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))])

In [13]:
class_rep(pipeline2, X_test, y_test)

related
              precision    recall  f1-score   support

           0       0.59      0.27      0.37      1564
           1       0.80      0.94      0.86      4947
           2       0.18      0.05      0.07        43

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.52      0.42      0.44      6554
weighted avg       0.74      0.77      0.74      6554

request
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      5445
           1       0.78      0.56      0.65      1109

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.85      0.76      0.80      6554
weighted avg       0.89      0.90      0.89      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6521
           1       0.12      0.03      0.05        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.56      0.51      0.52      655

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.29      0.07      0.11        75

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.64      0.53      0.55      6554
weighted avg       0.98      0.99      0.98      6554

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      0.99      0.98      6267
           1       0.33      0.08      0.13       287

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.64      0.54      0.55      6554
weighted avg       0.93      0.95      0.94      6554

weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      4735
           1       0.86      0.66      0.75      1819

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.87      0.81      0.83      6554
weighted avg       0.87      0.88    

In [14]:
parameters2 = {
        'tfidf__use_idf': (True, False),
        'clf__estimator__n_estimators': [50, 60, 70]
}

cv2 = GridSearchCV(pipeline2, param_grid = parameters2)
cv2

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...or=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=None))]),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'tfidf__use_idf': (True, False), 'clf__estimator__n_estimators': [50, 60, 70]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [15]:
cv2.fit(X_train, y_train)
class_rep(cv2, X_test, y_test)

c:\users\chris\appdata\local\programs\python\python37\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


related
              precision    recall  f1-score   support

           0       0.58      0.26      0.36      1564
           1       0.80      0.94      0.86      4947
           2       0.50      0.16      0.25        43

   micro avg       0.77      0.77      0.77      6554
   macro avg       0.63      0.45      0.49      6554
weighted avg       0.74      0.77      0.74      6554

request
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      5445
           1       0.80      0.56      0.66      1109

   micro avg       0.90      0.90      0.90      6554
   macro avg       0.86      0.77      0.80      6554
weighted avg       0.90      0.90      0.90      6554

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6521
           1       0.20      0.03      0.05        33

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.60      0.51      0.52      655

              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6479
           1       0.18      0.05      0.08        75

   micro avg       0.99      0.99      0.99      6554
   macro avg       0.59      0.53      0.54      6554
weighted avg       0.98      0.99      0.98      6554

other_infrastructure
              precision    recall  f1-score   support

           0       0.96      0.99      0.97      6267
           1       0.27      0.08      0.12       287

   micro avg       0.95      0.95      0.95      6554
   macro avg       0.62      0.54      0.55      6554
weighted avg       0.93      0.95      0.94      6554

weather_related
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      4735
           1       0.86      0.68      0.76      1819

   micro avg       0.88      0.88      0.88      6554
   macro avg       0.87      0.82      0.84      6554
weighted avg       0.88      0.88    

### 9. Export your model as a pickle file

In [16]:
with open('model.pkl', 'wb') as f:
    pickle.dump(cv2, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.